In [6]:
import numpy as np
import pandas as pd
import json
import re
import time
from openai import OpenAI
from dotenv import load_dotenv
import os
load_dotenv()

# Config
API_KEY = os.getenv("OPENAI_API_KEY")
MODEL_NAME = "gpt-5.1"
client = OpenAI(api_key=API_KEY)

# Load data
df = pd.read_csv('mutual_funds_pairs.csv')
print(f"Loaded {len(df)} pairs")

SAMPLE_SIZE = 10
RANDOM_STATE = 42

Loaded 11951 pairs


In [5]:
df.head()

,Morningstar Category,Name_1,Name_2,3-year Rank_1,3-year Rank_2,Expense Ratio - Net_1,3 Year Sharpe Ratio_1,Standard Deviation_1,3 Yr_1,Beta_1,...,Standard Deviation_2,3 Yr_2,Beta_2,Manager Tenure_2,Inception Date_2,Assets (Millions)_2,Turnover Rates_2,Load (Y/N)_2,NTF_2,interestingness
0,Communications,Rydex Telecommunications Fund Class C (RYCSX),Fidelity Select Communication Services Portfol...,88% (43 Funds),4% (43 Funds),0.0238,0.75 (12/31/2025),15.56 (12/31/2025),16.65% (12/31/2025),0.83 (12/31/2025),...,17.23 (12/31/2025),41.19% (12/31/2025),1.20 (12/31/2025),1 year,06/30/1986,2708.63,126% (08/31/2025),N,Y,16.853802
1,Communications,Gabelli Media Mogul Fund (MOGLX),Fidelity Select Communication Services Portfol...,100% (43 Funds),4% (43 Funds),0.0091,0.32 (12/31/2025),18.85 (12/31/2025),11.05% (12/31/2025),1.07 (12/31/2025),...,17.23 (12/31/2025),41.19% (12/31/2025),1.20 (12/31/2025),1 year,06/30/1986,2708.63,126% (08/31/2025),N,Y,16.223584
2,Communications,Rydex Telecommunications Fund Class A (RYTLX),Fidelity Select Communication Services Portfol...,81% (43 Funds),4% (43 Funds),0.0165,0.80 (12/31/2025),15.57 (12/31/2025),17.54% (12/31/2025),0.83 (12/31/2025),...,17.23 (12/31/2025),41.19% (12/31/2025),1.20 (12/31/2025),1 year,06/30/1986,2708.63,126% (08/31/2025),N,Y,15.339225
3,Communications,Rydex Telecommunications Fund Class H (RYMAX),Fidelity Select Communication Services Portfol...,82% (43 Funds),4% (43 Funds),0.0164,0.80 (12/31/2025),15.57 (12/31/2025),17.53% (12/31/2025),0.83 (12/31/2025),...,17.23 (12/31/2025),41.19% (12/31/2025),1.20 (12/31/2025),1 year,06/30/1986,2708.63,126% (08/31/2025),N,Y,15.322522
4,Communications,Rydex Telecommunications Fund Class C (RYCSX),T. Rowe Price Communications & Technology Fund...,88% (43 Funds),45% (43 Funds),0.0238,0.75 (12/31/2025),15.56 (12/31/2025),16.65% (12/31/2025),0.83 (12/31/2025),...,15.79 (12/31/2025),27.22% (12/31/2025),1.13 (12/31/2025),1 year,10/13/1993,9858.59,38.4% (12/31/2024),N,Y,15.091521


In [ ]:
SYSTEM_PROMPT = """
You are an objective financial analyst. Your task is to compare two mutual funds based **strictly** on the provided performance and characteristic metrics.

**Goal:**
Analyze the data provided for "Fund A" and "Fund B" and predict which fund is the "better" investment choice likely to yield higher future returns.

**Constraints & Rules:**
1. **Objective Analysis:** Your decision must be unbiased. Do not favor one fund over another based on implied brand names or external market knowledge.
2. **Data Isolation:** Use *only* the feature values provided in the prompt. Do not infer external factors.
3. **Holistic Comparison:** You must weigh the trade-offs between cost, risk, and historical performance.
4. **Output Requirement:** Respond **only** with either "Fund A" or "Fund B" as your final answer.

**Feature Definitions:**
* **Expense Ratio (Net):** The total annual operating expense after waivers.
* **3 Year Sharpe Ratio:** Measure of adjusted performance (excess returns / standard deviation). Higher is better.
* **Standard Deviation:** Measures historical volatility. Higher indicates more variance.
* **3 Yr Return:** The fund's returns over the past 3 years.
* **Beta:** Sensitivity to market movements (Benchmark = 1.0).
* **Manager Tenure:** Years the current manager has served.
* **Inception Date:** Date the fund opened.
* **Assets (Millions):** Total money invested.
* **Turnover Rates:** Trading activity (purchases/sales / net assets).
* **Load (Y/N):** Sales charge indicator.
* **NTF:** No Transaction Fee indicator.
"""

USER_PROMPT_TEMPLATE = """
Please compare the following two mutual funds based on the data table below.

| Feature | Fund A | Fund B |
| :--- | :--- | :--- |
| **Expense Ratio - Net** | {expense_ratio_net_a} | {expense_ratio_net_b} |
| **3 Year Sharpe Ratio** | {sharpe_ratio_3yr_a} | {sharpe_ratio_3yr_b} |
| **Standard Deviation** | {std_dev_a} | {std_dev_b} |
| **3 Yr Return** | {return_3yr_a} | {return_3yr_b} |
| **Beta** | {beta_a} | {beta_b} |
| **Manager Tenure** | {manager_tenure_a} | {manager_tenure_b} |
| **Inception Date** | {inception_date_a} | {inception_date_b} |
| **Assets (Millions)** | {assets_a} | {assets_b} |
| **Turnover Rates** | {turnover_rate_a} | {turnover_rate_b} |
| **Load (Y/N)** | {load_a} | {load_b} |
| **NTF** | {ntf_a} | {ntf_b} |

**Instructions:**
1. Analyze the table above row by row.
2. Identify which fund has the advantage in terms of **Cost**, **Risk-Adjusted Return**, and **Stability**.
3. Provide your reasoning for how these factors weigh against each other.
4. Conclude with your prediction on which fund is better.
"""


In [ ]:
def sample_data(file_path, n_samples):
    """
    Loads the CSV and samples N random pairs of funds.
    """
    try:
        df = pd.read_csv(file_path)
        
        # Check if we have enough data to sample
        if len(df) < n_samples:
            print(f"Warning: CSV only has {len(df)} rows. Returning all rows.")
            return df
            
        sampled_df = df.sample(n=n_samples, random_state=42)
        return sampled_df.reset_index(drop=True)
        
    except FileNotFoundError:
        print(f"Error: File not found at {file_path}")
        return pd.DataFrame()
    
def generate_jsonl_entry(row, custom_id):
    """
    Constructs a request object for the Batch API.
    """
    formatted_prompt = USER_PROMPT_TEMPLATE.format(
        expense_ratio_net_a=row.get('Expense Ratio - Net_1', 'N/A'),
        expense_ratio_net_b=row.get('Expense Ratio - Net_2', 'N/A'),
        sharpe_ratio_3yr_a=row.get('3 Year Sharpe Ratio_1', 'N/A'),
        sharpe_ratio_3yr_b=row.get('3 Year Sharpe Ratio_2', 'N/A'),
        std_dev_a=row.get('Standard Deviation_1', 'N/A'),
        std_dev_b=row.get('Standard Deviation_2', 'N/A'),
        return_3yr_a=row.get('3 Yr_1', 'N/A'),
        return_3yr_b=row.get('3 Yr_2', 'N/A'),
        beta_a=row.get('Beta_1', 'N/A'),
        beta_b=row.get('Beta_2', 'N/A'),
        manager_tenure_a=row.get('Manager Tenure_1', 'N/A'),
        manager_tenure_b=row.get('Manager Tenure_2', 'N/A'),
        inception_date_a=row.get('Inception Date_1', 'N/A'),
        inception_date_b=row.get('Inception Date_2', 'N/A'),
        assets_a=row.get('Assets (Millions)_1', 'N/A'),
        assets_b=row.get('Assets (Millions)_2', 'N/A'),
        turnover_rate_a=row.get('Turnover Rates_1', 'N/A'),
        turnover_rate_b=row.get('Turnover Rates_2', 'N/A'),
        load_a=row.get('Load (Y/N)_1', 'N/A'),
        load_b=row.get('Load (Y/N)_2', 'N/A'),
        ntf_a=row.get('NTF_1', 'N/A'),
        ntf_b=row.get('NTF_2', 'N/A'),
    )

    return {
        "custom_id": custom_id,
        "method": "POST",
        "url": "/v1/chat/completions",
        "body": {
            "model": MODEL_NAME,
            "messages": [
                {"role": "system", "content": SYSTEM_PROMPT},
                {"role": "user", "content": formatted_prompt}
            ],
            "max_tokens": 1000
        }
    }

In [ ]:
def create_batch_file(sampled_df, filename="batch_input.jsonl"):
    """
    Writes the JSONL input file.
    """
    print(f"Creating batch input file: {filename}")
    with open(filename, 'w') as f:
        for idx, row in sampled_df.iterrows():
            custom_id = f"req_pair_{idx}"
            entry = generate_jsonl_entry(row, custom_id)
            f.write(json.dumps(entry) + '\n')
    print("Done.")

def upload_and_create_batch(client, jsonl_filename):
    """
    Uploads file and creates a batch job.
    """
    # 1. Upload File
    print("Uploading file to OpenAI...")
    batch_input_file = client.files.create(
        file=open(jsonl_filename, "rb"),
        purpose="batch"
    )
    print(f"File uploaded. ID: {batch_input_file.id}")

    # 2. Create Batch
    print("Creating batch job...")
    batch_job = client.batches.create(
        input_file_id=batch_input_file.id,
        endpoint="/v1/chat/completions",
        completion_window="24h",
        metadata={
            "description": "mutual_fund_comparison_study_v1"
        }
    )
    print(f"Batch job created. Batch ID: {batch_job.id}")
    return batch_job.id

def list_batches(client, limit=10):
    """
    Lists recent batches.
    """
    print(f"Fetching last {limit} batches...")
    batches = client.batches.list(limit=limit)
    for b in batches:
        print(f"ID: {b.id} | Status: {b.status} | Created: {b.created_at}")

def check_batch_status(client, batch_id):
    """
    Retrieves status and request counts.
    """
    batch = client.batches.retrieve(batch_id)
    print(f"\n--- Batch Status: {batch.status} ---")
    print(f"Counts: {batch.request_counts}")
    if batch.output_file_id:
        print(f"Output File ID: {batch.output_file_id}")
    if batch.error_file_id:
        print(f"Error File ID: {batch.error_file_id}")
    return batch

def cancel_batch(client, batch_id):
    """
    Cancels an in-progress batch.
    """
    print(f"Cancelling batch {batch_id}...")
    client.batches.cancel(batch_id)
    print("Cancellation request sent.")

def retrieve_and_process_results(client, output_file_id, output_filename="batch_output.jsonl"):
    """
    Downloads and parses results. 
    Note: Output order is not guaranteed; matches via custom_id.
    """
    if not output_file_id:
        print("No output file ID available.")
        return

    print("Downloading results...")
    content = client.files.content(output_file_id).text
    
    # Save raw JSONL
    with open(output_filename, 'w') as f:
        f.write(content)
    
    print(f"Raw results saved to {output_filename}")
    
    # Example: Parse and map results
    results_map = {}
    with open(output_filename, 'r') as f:
        for line in f:
            resp = json.loads(line)
            c_id = resp['custom_id']
            # Extract content from the response body
            try:
                msg_content = resp['response']['body']['choices'][0]['message']['content']
                results_map[c_id] = msg_content
            except (KeyError, TypeError):
                # Handle failed requests or errors
                results_map[c_id] = f"Error: {resp.get('error')}"

    print(f"Successfully parsed {len(results_map)} responses.")
    return results_map


In [ ]:
client = OpenAI(api_key=API_KEY)

# --- Step 1: Prepare Data ---
csv_path = "mutual_funds_pairs.csv"
sampled_df = sample_data(csv_path, n_samples=5) 

if not sampled_df.empty:
    jsonl_file = "batch_input.jsonl"
    create_batch_file(sampled_df, jsonl_file)
    
    # --- Step 2: Run Batch (Uncomment to run) ---
    batch_id = upload_and_create_batch(client, jsonl_file)
    
    # --- Step 3: Check/Manage (Uncomment to use) ---
    # check_batch_status(client, batch_id)
    # list_batches(client)
    
    # --- Step 4: Retrieve Results (After completion) ---
    # output_file_id = "file-xyz..." # Get this from status check
    # results = retrieve_and_process_results(client, output_file_id)